# YZM304 Ödev 2: CNN Özellik Çıkarımı ve Sınıflandırma  
**Ders:** YZM304 Derin Öğrenme  
**Dönem:** Bahar 2025

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# GPU varsa kullan, yoksa CPU ile çalış
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Kullanılan cihaz:', device)

Kullanılan cihaz: cuda


## Veri Yükleme ve Önişleme  
MNIST veri setini normalize ederek yükle.

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

batch_size = 64
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader   = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

100.0%
100.0%
100.0%
100.0%


## Model Tanımları

In [3]:
# Model 1: LeNet-5
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16 * 53 * 53, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 53 * 53)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Model 2: BatchNorm ve Dropout eklenmiş LeNet-5
class LeNet5_BN_Drop(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1    = nn.Conv2d(1, 6, 5)
        self.bn1      = nn.BatchNorm2d(6)
        self.pool     = nn.MaxPool2d(2, 2)
        self.conv2    = nn.Conv2d(6, 16, 5)
        self.bn2      = nn.BatchNorm2d(16)
        self.fc1      = nn.Linear(16 * 53 * 53, 120)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2      = nn.Linear(120, 84)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3      = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 16 * 53 * 53)
        x = self.dropout1(torch.relu(self.fc1(x)))
        x = self.dropout2(torch.relu(self.fc2(x)))
        return self.fc3(x)

# Model 3: MNIST için uyarlanmış ön-eğitimli AlexNet
class PretrainedAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.model = torchvision.models.alexnet(pretrained=False)
        self.model.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
        self.model.classifier[6] = nn.Linear(4096, num_classes)

    def forward(self, x):
        return self.model(x)

# Model 5: Özel Derin CNN
class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1   = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2   = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3   = nn.Conv2d(64, 128, 3, padding=1)
        self.pool    = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1     = nn.Linear(128 * 28 * 28, 256)
        self.fc2     = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.dropout(torch.relu(self.fc1(x)))
        return self.fc2(x)

## Eğitim ve Değerlendirme Fonksiyonları

In [4]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(loader.dataset)

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    preds, true = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(labels.cpu().numpy())
    loss = running_loss / len(loader.dataset)
    acc = accuracy_score(true, preds)
    cm  = confusion_matrix(true, preds)
    return loss, acc, cm

## Model 1 Eğitimi ve Değerlendirme

In [5]:
num_epochs = 10
results = {}
model1 = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
for epoch in range(num_epochs):
    train_loss = train_epoch(model1, train_loader, criterion, optimizer1, device)
    val_loss, val_acc, _ = evaluate(model1, test_loader, criterion, device)
    print(f"Model1 Epoch {epoch+1}/{num_epochs} - Test Doğruluk: {val_acc:.4f}")
results['Model1'] = val_acc

Model1 Epoch 1/10 - Test Doğruluk: 0.9786
Model1 Epoch 2/10 - Test Doğruluk: 0.9826
Model1 Epoch 3/10 - Test Doğruluk: 0.9830
Model1 Epoch 4/10 - Test Doğruluk: 0.9809
Model1 Epoch 5/10 - Test Doğruluk: 0.9821
Model1 Epoch 6/10 - Test Doğruluk: 0.9843
Model1 Epoch 7/10 - Test Doğruluk: 0.9790
Model1 Epoch 8/10 - Test Doğruluk: 0.9801
Model1 Epoch 9/10 - Test Doğruluk: 0.9842
Model1 Epoch 10/10 - Test Doğruluk: 0.9803


## Model 2 Eğitimi ve Değerlendirme

In [6]:
model2 = LeNet5_BN_Drop().to(device)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
for epoch in range(num_epochs):
    train_loss = train_epoch(model2, train_loader, criterion, optimizer2, device)
    val_loss, val_acc, _ = evaluate(model2, test_loader, criterion, device)
    print(f"Model2 Epoch {epoch+1}/{num_epochs} - Test Doğruluk: {val_acc:.4f}")
results['Model2'] = val_acc

Model2 Epoch 1/10 - Test Doğruluk: 0.9091
Model2 Epoch 2/10 - Test Doğruluk: 0.9440
Model2 Epoch 3/10 - Test Doğruluk: 0.9500
Model2 Epoch 4/10 - Test Doğruluk: 0.9474
Model2 Epoch 5/10 - Test Doğruluk: 0.9309
Model2 Epoch 6/10 - Test Doğruluk: 0.9515
Model2 Epoch 7/10 - Test Doğruluk: 0.9518
Model2 Epoch 8/10 - Test Doğruluk: 0.9593
Model2 Epoch 9/10 - Test Doğruluk: 0.9646
Model2 Epoch 10/10 - Test Doğruluk: 0.9712


## Model 3 Eğitimi ve Değerlendirme

In [7]:
model3 = PretrainedAlexNet().to(device)
optimizer3 = optim.Adam(model3.parameters(), lr=0.001)
for epoch in range(num_epochs):
    train_loss = train_epoch(model3, train_loader, criterion, optimizer3, device)
    val_loss, val_acc, _ = evaluate(model3, test_loader, criterion, device)
    print(f"Model3 Epoch {epoch+1}/{num_epochs} - Test Doğruluk: {val_acc:.4f}")
results['Model3'] = val_acc

c:\Users\ERIS\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ERIS\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model3 Epoch 1/10 - Test Doğruluk: 0.9787
Model3 Epoch 2/10 - Test Doğruluk: 0.9817
Model3 Epoch 3/10 - Test Doğruluk: 0.9895
Model3 Epoch 4/10 - Test Doğruluk: 0.9869
Model3 Epoch 5/10 - Test Doğruluk: 0.9888
Model3 Epoch 6/10 - Test Doğruluk: 0.9878
Model3 Epoch 7/10 - Test Doğruluk: 0.9910
Model3 Epoch 8/10 - Test Doğruluk: 0.9907
Model3 Epoch 9/10 - Test Doğruluk: 0.9907
Model3 Epoch 10/10 - Test Doğruluk: 0.9904


Burada aldığım hata kodu etkilemediği ve üstteki sütun yaklaşık 15 dakikada çalıştığı için düzeltip yeniden çalıştırma gereği duymadım.

## Model 4: Hibrit CNN + SVM Özellik Çıkarımı

In [8]:
feature_extractor = nn.Sequential(
    model2.conv1, model2.bn1, nn.ReLU(), model2.pool,
    model2.conv2, model2.bn2, nn.ReLU(), model2.pool,
    nn.Flatten(),
    model2.fc1, nn.ReLU(),
    model2.fc2, nn.ReLU()
).to(device)
feature_extractor.eval()
train_feats, train_labels = [], []
with torch.no_grad():
    for inputs, lbls in train_loader:
        inputs = inputs.to(device)
        feats = feature_extractor(inputs)
        train_feats.append(feats.cpu().numpy())
        train_labels.append(lbls.numpy())
train_feats = np.concatenate(train_feats)
train_labels = np.concatenate(train_labels)
np.save('features.npy', train_feats)
np.save('labels.npy', train_labels)
svm = SVC()
svm.fit(train_feats, train_labels)
test_feats, test_labels = [], []
with torch.no_grad():
    for inputs, lbls in test_loader:
        inputs = inputs.to(device)
        feats = feature_extractor(inputs)
        test_feats.append(feats.cpu().numpy())
        test_labels.append(lbls.numpy())
test_feats  = np.concatenate(test_feats)
test_labels = np.concatenate(test_labels)
svm_preds = svm.predict(test_feats)
svm_acc = accuracy_score(test_labels, svm_preds)
print(f"Hibrit Model (SVM) Test Doğruluk: {svm_acc:.4f}")
results['Model4_SVM'] = svm_acc

Hibrit Model (SVM) Test Doğruluk: 0.9776


## Model 5 Eğitimi ve Değerlendirme

In [10]:
model5 = CustomCNN().to(device)
optimizer5 = optim.Adam(model5.parameters(), lr=0.001)
for epoch in range(num_epochs):
    train_loss = train_epoch(model5, train_loader, criterion, optimizer5, device)
    val_loss, val_acc, _ = evaluate(model5, test_loader, criterion, device)
    print(f"Model5 Epoch {epoch+1}/{num_epochs} - Test Doğruluk: {val_acc:.4f}")
results['Model5'] = val_acc

Model5 Epoch 1/10 - Test Doğruluk: 0.9802
Model5 Epoch 2/10 - Test Doğruluk: 0.9834
Model5 Epoch 3/10 - Test Doğruluk: 0.9856
Model5 Epoch 4/10 - Test Doğruluk: 0.9871
Model5 Epoch 5/10 - Test Doğruluk: 0.9845
Model5 Epoch 6/10 - Test Doğruluk: 0.9869
Model5 Epoch 7/10 - Test Doğruluk: 0.9868
Model5 Epoch 8/10 - Test Doğruluk: 0.9875
Model5 Epoch 9/10 - Test Doğruluk: 0.9876
Model5 Epoch 10/10 - Test Doğruluk: 0.9873


## Sonuçların Özeti

In [11]:
for name, acc in results.items():
    print(f"{name}: Test Doğruluk = {acc:.4f}")

Model1: Test Doğruluk = 0.9803
Model2: Test Doğruluk = 0.9712
Model3: Test Doğruluk = 0.9904
Model4_SVM: Test Doğruluk = 0.9776
Model5: Test Doğruluk = 0.9873
